### Fitting flat $\Lambda$CDM model to the SNIa database

- The aim of this code is to constrain flat $\Lambda$CDM model by fitting to the Pantheon+ SNIa database. The fitting is performed using an MCMC algorithm that minimizes the $\chi^2$ statistic based on the distance moduli calculated from the model and observed data. <br>
The fitting process consists of two steps: <br>
Step #1 - Finding outliers: With **SigmaClipping** set to True, outlier SNe are identified through automatic sigma clipping over multiple iterations. <br>
Step #2 - Constraining parameters: With **SigmaClipping** set to False, SN and model parameters are constrained using only the non-discarded SNe. <br>
<br>
<ins>Parameters:</ins> <br>
- **SigmaClipping**: True or False. <br>
$\quad$ $\quad$ True - Finding outliers, <br>
$\quad$ $\quad$ False - Constraining SN and model parameters. <br>
- **ParamNames**: List of the fitted parameters. The following parameters are fitted simultaneously: <br>
$\quad$ $\quad$ SN free parameters: $\alpha$, $\beta$, $\gamma$, and $M_B$ <br>
$\quad$ $\quad$ Model parameters: $h$ ($= H_0/100,\textrm{km},\textrm{s}^{-1},\textrm{Mpc}^{-1}$, the reduced Hubble constant), and $\Omega_{m,0}$ (the matter density parameter). <br>
- **Labels**: Formatted parameter names as they appear on the corner plot. <br>
- **Om_ind**: Index of $\Omega_{m,0}$ in **ParamNames**. <br>
- **parameter_limits**: Defines parameter space boundaries (column #1: lower, column #2: upper boundary) to be explored in the fitting process, as well as the step length (column #3) applied by the MCMC algorithm along the different dimensions of the parameter space. Each row corresponds to one fitted parameter, in the same order defined in **ParamNames**. <br>
- **TrialNum**: Number of MCMC steps (integer). Based on preliminary runs, 5 million steps consistently identify the same outliers, while 50 million steps consistently return the same p-values in the Anderson-Darling test. Therefore **TrialNum** is set to: <br>
$\quad$ $\quad$ 5,000,000 if **SigmaClipping** = True, <br>
$\quad$ $\quad$ 50,000,000 if **SigmaClipping** = False. <br>
- **printstep**: Displays the number of steps and elapsed time in seconds after every **printstep** steps in the MCMC process. **printstep** must be an integer. <br>
- **burn**: Burn-in period (integer). The first **burn** steps are discarded to determine optimal parameters. Based on preliminary runs, **burn** is set to 5,000 steps. <br>
- **bins**: Number of bins for the histograms on the corner plot. <br>
<br>
<ins>Calculated parameters:</ins> <br>
- **NumParams**: Number of fitted parameters, equal to the length of **ParamNames**. <br>
- **Prefix**: String used to distinguish different fitted models, here 'SNIa_FlatLCDM'. It is appended to the generated filenames. If **SigmaClipping** is True, **Prefix** is further extended with **sc _IterationNumber_**. <br>
<br>
<ins>Input files</ins> (Must be in the same folder as this code!): <br>
- **Pantheon+SH0ES.dat**: Pantheon+ SNIa database, stored in **SNIa_data**, where the entire database is restricted to the following columns: <br>
$\quad$ $\quad$ #3 : zHD - Hubble Diagram Redshift (with CMB and VPEC corrections), <br>
$\quad$ $\quad$ #13: CEPH_DIST - cepheid calculated absolute distance to host (uncertainty is incorporated in covariance matrix .cov), <br>
$\quad$ $\quad$ #14: IS_CALIBRATOR - binary to designate if this SN is in a host that has an associated cepheid distance, <br>
$\quad$ $\quad$ #20: mB - SALT2 uncorrected brightness, <br>
$\quad$ $\quad$ #18: x1 - SALT2 stretch, <br>
$\quad$ $\quad$ #16: c - SALT2 color, <br>
$\quad$ $\quad$ #35: HOST_LOGMASS - Host Galaxy Log Stellar Mass, <br>
$\quad$ $\quad$ #44: biasCor_m_b - Bias correction applied to brightness m_b <br>
$\color{red}{\text{Pantheon+ SNIa database is not included in this GitHub repository, please download it from}}$ https://github.com/PantheonPlusSH0ES/DataRelease/blob/main/Pantheon%2B_Data/4_DISTANCES_AND_COVAR/Pantheon%2BSH0ES.dat. <br>
- **Pantheon+SH0ES_STAT+SYS.cov**: Pantheon+ Statistical + Systematic Covariance matrix. Matrix formed data is stored in **Cov**, while its inverse in **Cov_inv**. <br>
$\color{red}{\text{Pantheon+ Statistical + Systematic Covariance matrix is not included in this GitHub repository, please download it from}}$ https://github.com/PantheonPlusSH0ES/DataRelease/blob/main/Pantheon%2B_Data/4_DISTANCES_AND_COVAR/Pantheon%2BSH0ES_STAT%2BSYS.cov <br>
<br>
<ins>Functions:</ins> <br>
- **Mu_measured**: Returns the distance moduli for each SNe (array **mu_meas**) calculated from the measurement data using the following parameters: <br>
$\quad$ $\quad$ mB, x1, co, m_host, d_bias : measured data, comes from the array **SNIa_data**, <br>
$\quad$ $\quad$ alpha, beta, gamma, MB : free SN parameters, <br>
$\quad$ $\quad$ tau : the step width applied when determining $\delta_{\textrm{host}}$. <br>
$\quad$ $\quad$ $\quad$ Source for $\tau$: https://github.com/PantheonPlusSH0ES/DataRelease/tree/main/Pantheon%2B_Data/5_COSMOLOGY/chains <br>
- **Integrate**: This function calculates the integral of the reciprocal of the expansion function, separated from **Mu_theory**. The input parameters are: <br>
$\quad$ $\quad$ z : redshift - measured data, comes from the array **SNIa_data**, <br>
$\quad$ $\quad$ Om : matter density parameter - free parameter of the model. <br>
- **Mu_theory**: Returns the distance moduli for each SNe (array **mu_th_calc**) calculated from the cosmological model using the following parameters: <br>
$\quad$ $\quad$ z : redshift - measured data, comes from the array **SNIa_data**, <br>
$\quad$ $\quad$ h : reduced Hubble constant - free parameter of the model, <br>
$\quad$ $\quad$ integr : result of **Integrate**, <br>
$\quad$ $\quad$ c : speed of light in vacuum expressed in m/s units - hardcoded parameter. <br>
$\quad$ For Cepheid-calibrated SNe (calib=1), the calculated distance modulus is replaced by the Cepheid distance modulus (mu_calib) in the **mu_th** array. <br>
- **ChiCalculator**: Calculates and returns $\chi^2$ for each SNe (array **Chi**) from **mu_meas**, **mu_th** and **Cov_inv**, according to eq.(6) in https://iopscience.iop.org/article/10.3847/2041-8213/ac5c5b. <br>
<br>
<ins>MCMC algorithm:</ins> <br>
- In the **MCMC** function we start from a randomly chosen point in parameter space, with coordinates drawn uniformly between parameter boundaries, and compute the initial $\chi^2$ statistic (**OutStat**). <br>
We then take **TrialNum** steps, each time calculating the $\chi^2$ statistic (**TestOutStat**) with updated parameters (**TestParams**). Each step involves randomly selecting a parameter (**ParamInd**) and choosing a step length from a Gaussian distribution with zero mean and sigma defined in **parameter_limits** (**RandStepParam**). If a step moves out of bounds, we adjust in the opposite direction. <br>
Steps are accepted with probability 1 if $\chi^2$ improves, or with an acceptance probability **a** if it worsens. Otherwise the new step is rejected and we keep the previous step's parameters. <br>
Each step is recorded in the file **OutMatrix_*Prefix*.txt**. <br>
To save computation time, the **integr** array is recalculated only if the chosen parameter is $\Omega_{m,0}$, otherwise, the array from the previously accepted step is used. <br>
<br>
<ins>Optimal parameter values:</ins> <br>
- **OptPar** function determines optimal parameters with left and right uncertainties as the 0.5, 0.16 and 0.84 quantiles. It processes values from **OutMatrix_*Prefix*.txt**, excluding the first **burn** steps. The result is written in a file named **params_*Prefix*.txt**. <br>
<br>
<ins>Sigma clipping:</ins> <br>
- If **SigmaClipping** is True, sigma clipping is applied automatically (function **Sigma_Clipping**), with multiple iterations per fitting process. After each iteration, SNe with distance moduli deviating more than 3 sigma from the model, based on the current parameters, are filtered out. <br>
<br>
<ins>RunThis:</ins> <br>
- If **SigmaClipping** is True, the function **RunThis** executes the entire iteration process by calling **MCMC**, **OptPar** and **Sigma_Clipping**. The string **sc** along with the current iteration number (**IterationNumber**) is appended to filenames. An automatically generated file **outfiltered_*Prefix*.txt** records the iteration number at which each SNe was discarded (or 0 if not rejected). The start and end of each iteration, along with the count of outfiltered SNe, are displayed on screen. The process continues until no more outliers are found. <br>
- If **SigmaClipping** is False, the function **RunThis** calls **MCMC** and **OptPar** using the non-discarded SNe only. <br>
<br>
<ins>Output files</ins> (Generated in the same location as the code.): <br>
- **OutMatrix_*Prefix*.txt**: MCMC output containing parameter values and $\chi^2$ statistics. It has # parameters + 1 columns and TrialNum + 1 rows (including the initial configuration), with a new line added after each step. During Step #1 when **SigmaClipping** is True, a new file is created for each iteration, distinguished by appending **sc _IterationNumber_** to the filename. In Step #2 one such file is created. <br>
- **outfiltered_*Prefix*.txt**: Matches the SNIa database in rows and records the IterationNumber when each SNIa was rejected. Only one such file is created during Step #1 when **SigmaClipping** is True, and this file is used in Step #2. <br>
- **params_*Prefix*.txt**: Contains parameter values (column #1) and uncertainties (columns #2: left, #3: right), with rows corresponding to parameters in the order defined by **ParamNames**. During Step #1 when **SigmaClipping** is True, a new file is created for each iteration, distinguished by appending **sc _IterationNumber_** to the filename. In Step #2 one such file is created. <br>
- ***Prefix*_posteriors.pdf**: Corner plot of the fitted parameters, generated at the conclusion of Step #2.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import time
import corner
from scipy.integrate import quad

#### Parameters

In [ ]:
SigmaClipping    = True
ParamNames       = ['alpha','beta','gamma','MB','h','Om']
Labels           = [r'$\alpha$',r'$\beta$',r'$\gamma$',r'$M_B$',r'$H_0$',r'$\Omega_{m,0}$']
Om_ind           = 5
parameter_limits = np.array([[0,0.2,0.01],[1.3,3.2,0.05],[-0.6,0.2,0.05],[-20,-18.8,0.05],[0.62,0.78,0.01],[0,1,0.05]])
TrialNum         = int((5 if SigmaClipping else 50) * 1e6)
printstep        = int(TrialNum/10)
burn             = 5000
bins             = 100
#############################################################################################
NumParams        = len(ParamNames)
Prefix           = 'SNIa_FlatLCDM'
print(Prefix)

#### Input files

In [ ]:
SNIa_data        = np.genfromtxt('Pantheon+SH0ES.dat', skip_header = True, delimiter=' ')[:,[2,12,13,19,17,15,34,43]]
Cov_raw          = np.genfromtxt('Pantheon+SH0ES_STAT+SYS.cov')
Cov              = Cov_raw[1:].reshape(int(Cov_raw[0]),int(Cov_raw[0]))
Cov_inv          = np.linalg.inv(Cov)
del Cov_raw

#### Functions

In [ ]:
c   = 299792458
tau = 0.0697186

def Mu_measured(SNIa_dat, Params):    
    [z, mu_calib, calib, mB, x1, co, m_host, d_bias] = [SNIa_dat[:,i] for i in range(len(SNIa_dat[0]))]     
    [alpha, beta, gamma, MB, h, Om]                  = [Params[i] for i in range(NumParams)]    
    d_host  = gamma * 1/(1 + np.exp((m_host-10)/tau)) - gamma/2    
    mu_meas = mB + alpha * x1 - beta * co - MB - d_bias + d_host    
    return mu_meas

def Integrate(z, Om):    
    def I(x):
        return 1 / np.sqrt( Om*(1+x)**3 + (1-Om) )    
    integr = np.empty(len(z))
    for i in range(len(z)):
        integr[i] = quad(I, 0, z[i])[0]    
    return integr

def Mu_theory(SNIa_dat, Params, integr):    
    [z, mu_calib, calib, mB, x1, co, m_host, d_bias] = [SNIa_dat[:,i] for i in range(len(SNIa_dat[0]))]
    [alpha, beta, gamma, MB, h, Om]                  = [Params[i] for i in range(NumParams)]    
    dL    = (1+z) * c/h * integr
    mu_th_calc = 5 * np.log10(dL)
    mu_th = mu_calib * calib + mu_th_calc * (1-calib)
    return mu_th, mu_th_calc

def ChiCalculator(SNIa_dat, Params, integr, Cov_inv):    
    mu_meas           = Mu_measured(SNIa_dat, Params)
    mu_th, mu_th_calc = Mu_theory(SNIa_dat, Params, integr)    
    res               = mu_meas - mu_th
    Chi               = res * np.matmul(Cov_inv, res)    
    return Chi

#### MCMC algorithm

In [ ]:
def MCMC(SNIa_dat, Covinv, Prefix):
    
    # Initial configuration
    Params        = np.empty(NumParams)
    for i in range(NumParams):
        Params[i] = parameter_limits[i][0] + (parameter_limits[i][1] - parameter_limits[i][0]) * random.random() 
    z             = SNIa_dat[:,0]
    integr        = Integrate(z, Params[Om_ind])
    OutStat       = sum(ChiCalculator(SNIa_dat, Params, integr, Covinv))
    Params_out    = np.append(Params, OutStat)
    np.savetxt('OutMatrix_' + Prefix + '.txt', Params_out, newline=' ') 
        
    # Wandering within the the parameter space for TrialNum steps
    for j in range(TrialNum):
        if j/printstep == np.floor(j/printstep):
            start = time.time()
        
        Test_Params   = Params.copy()
        ParamInd      = random.randint(1, NumParams)-1        
        RandStepParam = parameter_limits[ParamInd][2] * random.gauss(0,1)
        
        if parameter_limits[ParamInd][0] <= Test_Params[ParamInd]+RandStepParam <= parameter_limits[ParamInd][1]:
            Test_Params[ParamInd]   = Test_Params[ParamInd] + RandStepParam
        else: Test_Params[ParamInd] = Test_Params[ParamInd] - RandStepParam        
                
        if ParamInd == Om_ind:
            Test_integr   = Integrate(z, Test_Params[Om_ind])
        else: Test_integr = integr
        
        Test_OutStat = sum(ChiCalculator(SNIa_dat, Test_Params, Test_integr, Covinv))
   
        a = np.exp(-(Test_OutStat-OutStat)/2)
        p = random.random()
        if a >= p:
            OutStat  = Test_OutStat
            Params   = Test_Params
            integr   = Test_integr
            
        Params_out   = np.append(Params, OutStat)
        with open('OutMatrix_' + Prefix + '.txt','a') as f:
            f.write('\n')
            np.savetxt(f, Params_out, newline=' ')
                
        if (j+1)/printstep == np.floor((j+1)/printstep):
            end = time.time()
            print(j+1, end-start)

#### Optimal parameter values

In [ ]:
def OptPar(Prefix):
    
    OutMatrix = np.genfromtxt('OutMatrix_' + Prefix + '.txt', delimiter=' ')
    OutM      = OutMatrix[:,range(NumParams)][burn:]    
    quantiles = [0.16, 0.5, 0.84]
    
    OptParams = []
    for i in range(NumParams):
        opt       = corner.quantile(OutM[:,i],quantiles)
        OptParams = OptParams + [[opt[1], opt[1]-opt[0], opt[2]-opt[1]]]
    np.savetxt('params_' + Prefix + '.txt', OptParams, delimiter='\t')
    
    if SigmaClipping == False:
        OutM1      = OutM.copy()
        OutM1[:,4] = OutM1[:,4]*100
        figure = corner.corner(OutM1, bins=bins, labels=Labels, quantiles=quantiles, show_titles=True, title_fmt='.4f', title_kwargs={"fontsize": 12})
        figure.savefig(Prefix + '_posteriors.pdf')

#### Sigma clipping

In [ ]:
def Sigma_Clipping(Prefix, Prefix2, IterationNumber):
    
    Params            = np.genfromtxt('params_' + Prefix2 + '.txt')[:,0]
    mu_meas           = Mu_measured(SNIa_data, Params)
    z                 = SNIa_data[:,0]
    integr            = Integrate(z, Params[Om_ind])
    mu_th, mu_th_calc = Mu_theory(SNIa_data, Params, integr)
    res               = np.abs(mu_meas - mu_th_calc) / np.sqrt(Cov.diagonal())
    outf              = res > 3
    
    Outfiltered       = np.genfromtxt('outfiltered_' + Prefix + '.txt')
    for i in range(len(Outfiltered)):
        if (Outfiltered[i] == 0 and outf[i] == True):
            Outfiltered[i] = IterationNumber        
    np.savetxt('outfiltered_' + Prefix + '.txt', Outfiltered)
    
def Masking(Prefix):
    Outfiltered  = np.genfromtxt('outfiltered_' + Prefix + '.txt')
    mask         = Outfiltered == 0
    Data         = SNIa_data[mask]
    Cov_masked   = np.asarray([Cov[i][mask] for i in range(len(Cov))])[mask]
    Covinv       = np.linalg.inv(Cov_masked)
    return Data, Covinv

#### RunThis

In [ ]:
def RunThis():
    
    if SigmaClipping:
        IterationNumber = 0
        OutfiltCount    = 1
    
        while OutfiltCount > 0:
        
            IterationNumber += 1
            print('Start iteration #' + str(IterationNumber))            
            Prefix2 = Prefix + '_sc' + str(IterationNumber)
        
            if IterationNumber == 1:
                np.savetxt('outfiltered_' + Prefix + '.txt', [0 for i in range(len(SNIa_data))])

            Data, Covinv = Masking(Prefix)  
            MCMC(Data, Covinv, Prefix2)
            OptPar(Prefix2)
            Sigma_Clipping(Prefix, Prefix2, IterationNumber)
        
            Outfiltered  = np.genfromtxt('outfiltered_' + Prefix + '.txt')
            OutfiltCount = sum(Outfiltered == IterationNumber)
            print('End iteration #' + str(IterationNumber) + '; Outfiltered: ' + str(OutfiltCount))
            
    else:
        Data, Covinv = Masking(Prefix)
        MCMC(Data, Covinv, Prefix)
        OptPar(Prefix)

In [ ]:
RunThis()